# Import libraries

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from fuzzywuzzy import process, fuzz
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

When we first tried to load the csv file into a dataframe, we got this error 
ParserError: Error tokenizing data. C error: Expected 12 fields in line 3350, saw 13

Meaning that at row 3350, there is an extra column. We modified the file trough excel where we've noticed that the columns were shifted to the right. The same for the rows 4704,5879 and 8981  

In [3]:
df = pd.read_csv(r"C:\Users\HP\OneDrive\Desktop\DSTI\Python Machine Learning Labs\Project-20221212\books.csv")

# Exploratory Data Analysis

In [4]:
#Columns type
df.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                float64
language_code          object
  num_pages             int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
dtype: object

In [5]:
#Is there any NA values in the dataset?
df.isna().sum()

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

In [6]:
#Exploring statistical summary for numerical columns and some information for non-numerical columns 
df.describe(include = "all")

bookID      title         authors  average_rating        isbn  \
count   11127.000000      11127           11127    11127.000000       11127   
unique           NaN      10352            6643             NaN       11127   
top              NaN  The Iliad  P.G. Wodehouse             NaN  1565482344   
freq             NaN          9              40             NaN           1   
mean    21310.938887        NaN             NaN        3.933631         NaN   
std     13093.358023        NaN             NaN        0.352445         NaN   
min         1.000000        NaN             NaN        0.000000         NaN   
25%     10287.000000        NaN             NaN        3.770000         NaN   
50%     20287.000000        NaN             NaN        3.960000         NaN   
75%     32104.500000        NaN             NaN        4.135000         NaN   
max     45641.000000        NaN             NaN        5.000000         NaN   

              isbn13 language_code     num_pages  ratings_count  \
count   1.112700e+04         11127  11127.000000   1.112700e+04   
unique           NaN            27           NaN            NaN   
top              NaN           eng           NaN            NaN   
freq             NaN          8911           NaN            NaN   
mean    9.759888e+12           NaN    336.376921   1.793641e+04   
std     4.428964e+11           NaN    241.127305   1.124794e+05   
min     8.987060e+09           NaN      0.000000   0.000000e+00   
25%     9.780350e+12           NaN    192.000000   1.040000e+02   
50%     9.780590e+12           NaN    299.000000   7.450000e+02   
75%     9.780870e+12           NaN    416.000000   4.993500e+03   
max     9.790010e+12           NaN   6576.000000   4.597666e+06   

        text_reviews_count publication_date publisher  
count         11127.000000            11127     11127  
unique                 NaN             3678      2292  
top                    NaN         10-01-05   Vintage  
freq                   NaN               56       318  
mean            541.854498              NaN       NaN  
std            2576.176608              NaN       NaN  
min               0.000000              NaN       NaN  
25%               9.000000              NaN       NaN  
50%              46.000000              NaN       NaN  
75%             237.500000              NaN       NaN  
max           94265.000000              NaN       NaN

In [7]:
#What are the books that have 0 pages?  76 of them
pages = df['  num_pages']
df[pages == 0]

bookID                                              title  \
306       955  The 5 Love Languages / The 5 Love Languages Jo...   
853      2835                   The Tragedy of Pudd'nhead Wilson   
1061     3593  Murder by Moonlight & Other Mysteries (New Adv...   
1064     3599  The Unfortunate Tobacconist & Other Mysteries ...   
1230     4249             The Da Vinci Code (Robert Langdon  #2)   
...       ...                                                ...   
10030   40378                  The Chessmen of Mars (Barsoom #5)   
10192   41273                     Fine Lines (One-Eyed Mack  #6)   
10624   43343  Stowaway and Milk Run: Two Unabridged Stories ...   
10926   44748                        The Mask of the Enchantress   
11081   45472  Treasury of American Tall Tales: Volume 1: Dav...   

                                                 authors  average_rating  \
306                                         Gary Chapman            4.70   
853                          Mark Twain/Michael Prichard            3.79   
1061                                          NOT A BOOK            4.00   
1064                                          NOT A BOOK            3.50   
1230                              Dan Brown/Paul Michael            3.84   
...                                                  ...             ...   
10030                    Edgar Rice Burroughs/John Bolen            3.83   
10192                                         Jim Lehrer            3.23   
10624                     Mary Higgins Clark/Jan Maxwell            3.49   
10926                                      Victoria Holt            3.85   
11081  David Bromberg/Jay Ungar/Molly Mason/Garrison ...            3.86   

             isbn        isbn13 language_code    num_pages  ratings_count  \
306     802415318  9.780800e+12           eng            0             22   
853    140015068X  9.781400e+12           eng            0              3   
1061    743564677  9.780740e+12           eng            0              7   
1064   074353395X  9.780740e+12           eng            0             12   
1230    739339788  9.780740e+12           eng            0             91   
...           ...           ...           ...          ...            ...   
10030  1400130212  9.781400e+12           eng            0           5147   
10192   517164353  9.780520e+12           eng            0             17   
10624   671046241  9.780670e+12           eng            0             64   
10926   449210847  9.780450e+12           eng            0             21   
11081   739336509  9.780740e+12           eng            0             36   

       text_reviews_count publication_date                      publisher  
306                     4         01-01-05               Moody Publishers  
853                     0         01-01-03                   Tantor Media  
1061                    2         10-03-06          Simon  Schuster Audio  
1064                    1         10-01-03         Simon & Schuster Audio  
1230                   16        3/28/2006             Random House Audio  
...                   ...              ...                            ...  
10030                 157         01-01-05                   Tantor Media  
10192                   4       11/19/1995  Random House Value Publishing  
10624                   2         12-01-99         Simon & Schuster Audio  
10926                   1         10-12-81                      Ivy Books  
11081                   9        8/22/2006      Listening Library (Audio)  

[76 rows x 12 columns]

!! What does it mean NOT A BOOK in the authors column? bookID:3593, 3599

# Data Cleaning  

In [8]:
#Renaming '  num_pages' in order to remove the extra spaces and displaying columns names
df.rename(columns={'  num_pages':'num_pages'}, inplace=True)

For the language_code column, replacing the value eng-US, en-CA and eng-GB with the value eng 

In [9]:
df['language_code'] = df['language_code'].replace('en-US','eng')
df['language_code'] = df['language_code'].replace('en-GB','eng')
df['language_code'] = df['language_code'].replace('en-CA','eng')

In [10]:
df

bookID                                              title  \
0           1  Harry Potter and the Half-Blood Prince (Harry ...   
1           2  Harry Potter and the Order of the Phoenix (Har...   
2           4  Harry Potter and the Chamber of Secrets (Harry...   
3           5  Harry Potter and the Prisoner of Azkaban (Harr...   
4           8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
...       ...                                                ...   
11122   45631   Expelled from Eden: A William T. Vollmann Reader   
11123   45633                        You Bright and Risen Angels   
11124   45634                    The Ice-Shirt (Seven Dreams #1)   
11125   45639                                        Poor People   
11126   45641                        Las aventuras de Tom Sawyer   

                                                 authors  average_rating  \
0                             J.K. Rowling/Mary GrandPré            4.57   
1                             J.K. Rowling/Mary GrandPré            4.49   
2                                           J.K. Rowling            4.42   
3                             J.K. Rowling/Mary GrandPré            4.56   
4                             J.K. Rowling/Mary GrandPré            4.78   
...                                                  ...             ...   
11122  William T. Vollmann/Larry McCaffery/Michael He...            4.06   
11123                                William T. Vollmann            4.08   
11124                                William T. Vollmann            3.96   
11125                                William T. Vollmann            3.72   
11126                                         Mark Twain            3.91   

             isbn        isbn13 language_code  num_pages  ratings_count  \
0       439785960  9.780440e+12           eng        652        2095690   
1       439358078  9.780440e+12           eng        870        2153167   
2       439554896  9.780440e+12           eng        352           6333   
3      043965548X  9.780440e+12           eng        435        2339585   
4       439682584  9.780440e+12           eng       2690          41428   
...           ...           ...           ...        ...            ...   
11122  1560254416  9.781560e+12           eng        512            156   
11123   140110879  9.780140e+12           eng        635            783   
11124   140131965  9.780140e+12           eng        415            820   
11125    60878827  9.780060e+12           eng        434            769   
11126  8497646983  9.788500e+12           spa        272            113   

       text_reviews_count publication_date        publisher  
0                   27591        9/16/2006  Scholastic Inc.  
1                   29221         09-01-04  Scholastic Inc.  
2                     244         11-01-03       Scholastic  
3                   36325         05-01-04  Scholastic Inc.  
4                     164        9/13/2004       Scholastic  
...                   ...              ...              ...  
11122                  20       12/21/2004    Da Capo Press  
11123                  56         12-01-88    Penguin Books  
11124                  95         08-01-93    Penguin Books  
11125                 139        2/27/2007             Ecco  
11126                  12        5/28/2006    Edimat Libros  

[11127 rows x 12 columns]

## Publisher columns, many publishers have different spellings

In [11]:
pub = df.publisher.unique()
elements = list(pub)
# elements = ['vikash', 'vikas', 'Vinod', 'Vikky', 'Akash', 'Vinodh', 'Sachin', 'Salman', 'Ajay', 'Suchin', 'vIkas', 'salman', 'salMa', 'Akash', 'vikahs']

results = [[name, [], 0] for name in elements]

for (i, element) in enumerate(elements):
    for (j, choice) in enumerate(elements[i+1:]):
        if fuzz.ratio(element, choice) >= 90:
            results[i][2] += 1
            results[i][1].append(choice)
            results[j+i+1][2] += 1
            results[j+i+1][1].append(element)

data = pd.DataFrame(results, columns=['name', 'duplicates', 'duplicate_count'])

data = data.sort_values(by=['duplicate_count'],ascending=False)


In [12]:
to_remove = []
test = {}

for i,j in data.iterrows():
    if data.name[i] in list(test.keys()):
        for x in j.duplicates:
            if x not in test : 
                test[x]=test[data.name[i]]
    else : 
        test[data.name[i]] = data.name[i]
        for x in j.duplicates:
            if x not in test : 
                test[x]=data.name[i]
#                 print(data.name[i], "****", x,"--" , data.name[i])
        

                    
# print(test)
data['pub'] = data['name'].map(test)
display(data)

name  \
102          W. W. Norton & Company   
815            W.W. Norton  Company   
120           W. W. Norton  Company   
1722        Farrar  Straus & Giroux   
1419       W. W. Norton and Company   
...                             ...   
825               Albury Publishing   
824     Little  Brown Young Readers   
823   Random House Value Publishing   
822                   Holt McDougal   
2291             Editorial Presença   

                                             duplicates  duplicate_count  \
102   [W. W. Norton  Company, W.W. Norton & Company,...                4   
815   [W. W. Norton & Company, W. W. Norton  Company...                4   
120   [W. W. Norton & Company, W.W. Norton & Company...                4   
1722  [Farrar  Straus and Giroux, Farrar Straus Giro...                3   
1419  [W. W. Norton & Company, W. W. Norton  Company...                3   
...                                                 ...              ...   
825                                                  []                0   
824                                                  []                0   
823                                                  []                0   
822                                                  []                0   
2291                                                 []                0   

                                pub  
102          W. W. Norton & Company  
815          W. W. Norton & Company  
120          W. W. Norton & Company  
1722        Farrar  Straus & Giroux  
1419         W. W. Norton & Company  
...                             ...  
825               Albury Publishing  
824     Little  Brown Young Readers  
823   Random House Value Publishing  
822                   Holt McDougal  
2291             Editorial Presença  

[2292 rows x 4 columns]

In [18]:
len(df.publisher_iso.unique())

2143

In [13]:
df['publisher_iso'] = df['publisher'].map(test)

In [19]:
df

bookID                                              title  \
0           1  Harry Potter and the Half-Blood Prince (Harry ...   
1           2  Harry Potter and the Order of the Phoenix (Har...   
2           4  Harry Potter and the Chamber of Secrets (Harry...   
3           5  Harry Potter and the Prisoner of Azkaban (Harr...   
4           8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
...       ...                                                ...   
11122   45631   Expelled from Eden: A William T. Vollmann Reader   
11123   45633                        You Bright and Risen Angels   
11124   45634                    The Ice-Shirt (Seven Dreams #1)   
11125   45639                                        Poor People   
11126   45641                        Las aventuras de Tom Sawyer   

                                                 authors  average_rating  \
0                             J.K. Rowling/Mary GrandPré            4.57   
1                             J.K. Rowling/Mary GrandPré            4.49   
2                                           J.K. Rowling            4.42   
3                             J.K. Rowling/Mary GrandPré            4.56   
4                             J.K. Rowling/Mary GrandPré            4.78   
...                                                  ...             ...   
11122  William T. Vollmann/Larry McCaffery/Michael He...            4.06   
11123                                William T. Vollmann            4.08   
11124                                William T. Vollmann            3.96   
11125                                William T. Vollmann            3.72   
11126                                         Mark Twain            3.91   

             isbn        isbn13 language_code  num_pages  ratings_count  \
0       439785960  9.780440e+12           eng        652        2095690   
1       439358078  9.780440e+12           eng        870        2153167   
2       439554896  9.780440e+12           eng        352           6333   
3      043965548X  9.780440e+12           eng        435        2339585   
4       439682584  9.780440e+12           eng       2690          41428   
...           ...           ...           ...        ...            ...   
11122  1560254416  9.781560e+12           eng        512            156   
11123   140110879  9.780140e+12           eng        635            783   
11124   140131965  9.780140e+12           eng        415            820   
11125    60878827  9.780060e+12           eng        434            769   
11126  8497646983  9.788500e+12           spa        272            113   

       text_reviews_count publication_date  month    year        publisher  \
0                   27591       2006-09-16    9.0  2006.0  Scholastic Inc.   
1                   29221       2004-09-01    9.0  2004.0  Scholastic Inc.   
2                     244       2003-11-01   11.0  2003.0       Scholastic   
3                   36325       2004-05-01    5.0  2004.0  Scholastic Inc.   
4                     164       2004-09-13    9.0  2004.0       Scholastic   
...                   ...              ...    ...     ...              ...   
11122                  20       2004-12-21   12.0  2004.0    Da Capo Press   
11123                  56       1988-12-01   12.0  1988.0    Penguin Books   
11124                  95       1993-08-01    8.0  1993.0    Penguin Books   
11125                 139       2007-02-27    2.0  2007.0             Ecco   
11126                  12       2006-05-28    5.0  2006.0    Edimat Libros   

          publisher_iso  
0      Scholastic  Inc.  
1      Scholastic  Inc.  
2            Scholastic  
3      Scholastic  Inc.  
4            Scholastic  
...                 ...  
11122     Da Capo Press  
11123     Penguin Books  
11124     Penguin Books  
11125              Ecco  
11126     Edimat Libros  

[11125 rows x 15 columns]

## Date Column

In [14]:
# Changing the publication_date colummn type to a date format 
df['publication_date'] = pd.to_datetime(df['publication_date'], dayfirst = False, yearfirst = False, errors = 'coerce')

In [15]:
# Splitting date column into month and year columns 
df['month'] = df['publication_date'].dt.month
df['year'] = df['publication_date'].dt.year

In [16]:
df = df.reindex(columns=['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', 'num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'month', 'year','publisher', 'publisher_iso'])
df

bookID                                              title  \
0           1  Harry Potter and the Half-Blood Prince (Harry ...   
1           2  Harry Potter and the Order of the Phoenix (Har...   
2           4  Harry Potter and the Chamber of Secrets (Harry...   
3           5  Harry Potter and the Prisoner of Azkaban (Harr...   
4           8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
...       ...                                                ...   
11122   45631   Expelled from Eden: A William T. Vollmann Reader   
11123   45633                        You Bright and Risen Angels   
11124   45634                    The Ice-Shirt (Seven Dreams #1)   
11125   45639                                        Poor People   
11126   45641                        Las aventuras de Tom Sawyer   

                                                 authors  average_rating  \
0                             J.K. Rowling/Mary GrandPré            4.57   
1                             J.K. Rowling/Mary GrandPré            4.49   
2                                           J.K. Rowling            4.42   
3                             J.K. Rowling/Mary GrandPré            4.56   
4                             J.K. Rowling/Mary GrandPré            4.78   
...                                                  ...             ...   
11122  William T. Vollmann/Larry McCaffery/Michael He...            4.06   
11123                                William T. Vollmann            4.08   
11124                                William T. Vollmann            3.96   
11125                                William T. Vollmann            3.72   
11126                                         Mark Twain            3.91   

             isbn        isbn13 language_code  num_pages  ratings_count  \
0       439785960  9.780440e+12           eng        652        2095690   
1       439358078  9.780440e+12           eng        870        2153167   
2       439554896  9.780440e+12           eng        352           6333   
3      043965548X  9.780440e+12           eng        435        2339585   
4       439682584  9.780440e+12           eng       2690          41428   
...           ...           ...           ...        ...            ...   
11122  1560254416  9.781560e+12           eng        512            156   
11123   140110879  9.780140e+12           eng        635            783   
11124   140131965  9.780140e+12           eng        415            820   
11125    60878827  9.780060e+12           eng        434            769   
11126  8497646983  9.788500e+12           spa        272            113   

       text_reviews_count publication_date  month    year        publisher  \
0                   27591       2006-09-16    9.0  2006.0  Scholastic Inc.   
1                   29221       2004-09-01    9.0  2004.0  Scholastic Inc.   
2                     244       2003-11-01   11.0  2003.0       Scholastic   
3                   36325       2004-05-01    5.0  2004.0  Scholastic Inc.   
4                     164       2004-09-13    9.0  2004.0       Scholastic   
...                   ...              ...    ...     ...              ...   
11122                  20       2004-12-21   12.0  2004.0    Da Capo Press   
11123                  56       1988-12-01   12.0  1988.0    Penguin Books   
11124                  95       1993-08-01    8.0  1993.0    Penguin Books   
11125                 139       2007-02-27    2.0  2007.0             Ecco   
11126                  12       2006-05-28    5.0  2006.0    Edimat Libros   

          publisher_iso  
0      Scholastic  Inc.  
1      Scholastic  Inc.  
2            Scholastic  
3      Scholastic  Inc.  
4            Scholastic  
...                 ...  
11122     Da Capo Press  
11123     Penguin Books  
11124     Penguin Books  
11125              Ecco  
11126     Edimat Libros  

[11127 rows x 15 columns]

In [1]:
# bookID: 31373 where the date value before converting the column into a datetime type was 11/31/2000 (30 days for November)
# bookID: 45531 where the date value before converting the column into a datetime type was 6/31/1982 (30 days for June)
# Dropping those 2 rows 
df = df.drop(index=df.loc[df.publication_date.isna(),:].index)

NameError: name 'df' is not defined

# Next Steps

1. In the correlation section
   * Avg rating // language (visual)
   * Avg rating for top published authors over time (year) (bof)
   * Avg rating for top publishers over time (bof)
   * Corr month publication // avg rating / rating counts / text review
   * Corr year publication // avg rating / rating counts / text review
   

2. jointplot for text_reviews_count and ratings_count --> no histogram 

3. After dropping all outliers, how does the heatmap look like? 

4. Transforming categorical columns into numerical columns 
    * Title
    * Authors 
    * Language 
    * Publisher 

5. New column that identifies if the book has been written by a single or multiple authors
    Or trying to select only the first author of the book 

6. Identify similar publishers ??? 

7. Dropping the rows where the avg rating is 0

8. Dropping the rows where rating counts and text review counts are set to 0